<a href="https://colab.research.google.com/github/wajidminhas/AI-Agents/blob/main/RAG-PINECONE/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [ ]:
import getpass, time, os
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api  = userdata.get("PINECONE-API")

pc = Pinecone(api_key = pinecone_api)

In [ ]:
import time

index_name = "test-index-feb-28"  # change if desired

# Check if the index already exists
if index_name in pc.list_indexes().names(): # Added () to call the .names property
    print(f"Index '{index_name}' already exists. Connecting to existing index.")
    index = pc.Index(index_name)
else:
    print(f"Index '{index_name}' does not exist. Creating new index.")
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    index = pc.Index(index_name)

Index 'test-index-feb-28' already exists. Connecting to existing index.


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vector = embeddings.embed_query("going to build rag")

In [ ]:
vector[:4]

[0.03360629826784134,
 -0.012450501322746277,
 -0.032498568296432495,
 -0.024020353332161903]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

#Add items to vector store

 can add items to our vector store by using the add_documents function

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)


In [ ]:
document_1

Document(metadata={'source': 'tweet'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')

In [ ]:
from uuid import uuid4
uuid4()

UUID('e6aeceb1-52fa-40f5-a8bf-c7c73fa3cad0')

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['f4f7ab2a-6b93-49a5-8c66-5b17912b5d3e',
 'd9c5dee5-89e4-4ccc-8cd3-3cd181a1c0a1',
 '8b4322bc-d1f9-4763-9335-834390dee601',
 '1147e0fc-9dd9-4271-8dad-931c59d1fe56',
 'c63c9574-a1ec-4de6-9601-1e2638ac0c47',
 'bab1d72c-ab10-4b9d-98a3-2b296dbb6835',
 'b93c62f9-554a-484b-8edf-10f2bb70b2e2',
 'edcccfc4-1860-4bd0-972a-899e6aaa37f9',
 '79ec486b-f04a-467a-9333-eb04efaf99e5',
 '83f32e32-2825-43ef-876b-dee9c5e5cb78']

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",

)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "future of agentic ai? ",
)

for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{score}]")



* [SIM=0.614063] LangGraph is the best framework for building stateful, agentic applications! [0.614063203]
* [SIM=0.614063] LangGraph is the best framework for building stateful, agentic applications! [0.614063203]
* [SIM=0.614063] LangGraph is the best framework for building stateful, agentic applications! [0.614063203]
* [SIM=0.536725] Building an exciting new project with LangChain - come check it out! [0.536725223]
